In [90]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import random

In [92]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)

def get_html(url):
    try:
        random_sleep()
        r = requests.get(url)
    except:
        random_sleep(1000, 500)
        r = requests.get(url)
        return r.text
    return r.text

# получаем ссылки на все страницы
def get_total_pages(html, x=0):
    soup = BeautifulSoup(html, 'lxml')
    last_page_html = soup.find_all('a', class_='pagination-page')[-1].get('href')
    num_last_page = last_page_html.split('=')[1].split('&')[0]
    
    all_urls = ['https://www.avito.ru' + last_page_html.split('=')[0] + '=' \
                + str(i) + '&radius=0&f=188_893b0&i=1' \
                for i in range(x+1, int(num_last_page))]
    return all_urls
    
# получаем все ссылки на обхъявления с одной страницы
def get_page_urls(html):
    soup = BeautifulSoup(html, 'lxml')
    sp_ads = soup.find_all('a', class_='item-description-title-link')
    sleep(1)
    ads = [i.get('href') for i in sp_ads]
    urls = ['https://www.avito.ru' + str(i) for i in ads]
    return urls

# получаем данные одного объявления
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    params_html = soup.find('div', class_='item-params').find_all('li', class_='item-params-list-item')
    
    params = [i.text.strip() for i in params_html]
    price_html = soup.find('span', class_='js-item-price')
    price = 'Цена: ' + price_html.text.strip()
    params.append(price)
    return params

# получаем все даныне об объявлениях одной страниы
def get_all_page_params(html):
    urls = get_page_urls(html)
    parameters = []
    for url in urls:
        html = get_html(url)
        param = get_page_data(html)
        parameters.append(param)
    return parameters

# надо вставить url 2ой страницы из авито
# получаем массив из строк для всех страниц всех обхявлений
def get_all_data(url, x=0, num=4000):
    html_for_total_pages = get_html(url)
    total_pages_urls = get_total_pages(html_for_total_pages, x)
    sleep(10)
    all_data = []
    
    num_ads = 0 # считать сколько обхявлений скачалось
    num_pages = 0
    for page_url in total_pages_urls:
        try:
            page_html = get_html(page_url)
            adds_urls = get_page_urls(page_html)
        except:
            print("объявлений/страниц: ", num_ads," ", num_pages)
            return all_data
        num_pages += 1
        
        page_data = []

        random_sleep(25, 10)
        for ad_url in adds_urls:
            try:
                ad_html = get_html(ad_url)
                random_sleep(8, 8)
                one_data = get_page_data(ad_html)
            except:
                print("объявлений/страниц: ", num_ads," ", num_pages)
                all_data.extend(page_data)
                return all_data
            num_ads += 1
            if (num_ads == num):
                print("объявлений/страниц: ", num_ads," ", num_pages)
                all_data.extend(page_data)
                return all_data
            
            if (num_ads % 50 == 0):
                random_sleep(10, 20)
            if (num_ads % 200 == 0):
                random_sleep(20,40)
                
            url_of_data = 'Ссылка: ' + ad_url
            one_data.append(url_of_data)
            page_data.append(one_data)
        all_data.extend(page_data)
        
    return all_data

In [15]:
# делаем из массива массивов строк dataframe
def get_pd_data(data):
    df = pd.DataFrame(index=np.arange(len(data)), columns=['Марка', 'Модель', 'Год выпуска', 'Тип кузова',
                                                   'Цена', 'Объём двигателя', 'Цвет', 'Привод',
                                                   'Руль', 'Тип двигателя', 'Состояние', 'Владельцев по ПТС',
                                                   'Количество дверей', 'Мощность двигателя', 'Пробег',
                                                   'VIN или номер кузова', 'Коробка передач', 'Ссылка'])
    for j in range(len(data)):
        for i in range(len(data[j])):
            tmp = data[j][i].split(': ')
            df.loc[j][tmp[0]] = tmp[1]

    return df


In [1]:
url = 'https://www.avito.ru/moskva/avtomobili/ford?p=2&radius=0&f=188_893b0&i=1'
list_of_data = get_all_data(url)
df = get_pd_data(list_of_data)
df.to_csv('ford1.csv')

In [ ]:
random_sleep(300, 300)

In [35]:
url = 'https://www.avito.ru/moskva/avtomobili/hyundai?p=2&radius=0&f=188_893b0&i=1'
list_of_data = get_all_data(url)
df = get_pd_data(list_of_data)
df.to_csv('hyundai1.csv')

In [ ]:
random_sleep(300, 300)

In [2]:
url = 'https://www.avito.ru/moskva/avtomobili/mercedes-benz?p=2&radius=0&f=188_893b0&i=1'
list_of_data = get_all_data(url)
df = get_pd_data(list_of_data)
df.to_csv('mercedes-benz1.csv')

In [3]:
url = 'https://www.avito.ru/moskva/avtomobili/volkswagen?p=2&radius=0&f=188_893b0&i=1'
list_of_data = get_all_data(url)
df = get_pd_data(list_of_data)
df.to_csv('volkswagen1.csv')